# 2.9 大模型应用生产实践

## 🚄 前言

通过前面课程学习，你的答疑机器人已经可以比较好的回答私域知识，接下来将继续探讨如何将这个大模型应用发布至生产环境，即如何将大模型应用从开发和测试阶段转移到实际业务场景中，这是一个复杂且关键的过程，涉及多个步骤和技术考虑。本节课程将讨论它们帮助你理解并顺利完成这一过程。

## 🍁 课程目标

学完本课程后，你将能够：

* 通过业务需求分析明确将大模型应用发布至生产环境的关键要素
* 了解如何平衡大模型应用的性能和运行成本
* 了解如何提升大模型应用的稳定性

将大模型发布至生产环境并应用于实际业务场景并非易事。这不仅需要从业务场景出发，通过明确的需求选择更适合的模型（模型的功能性需求，如针对数学问题优化的Qwen-Math）。还需要从系统架构的角度全面考量性能、成本、安全性和稳定性等非功能性需求，模型的非功能性需求虽然不直接关联到具体功能，但对模型的整体质量和用户体验至关重要。

总体来说，功能性需求定义了大模型“做什么”，而非功能性需求则确保其在执行这些功能时“如何做”才能达到高质量标准，只有在业务需求和技术实现之间找到平衡，才能高效部署和运营大模型服务。

本节课程将围绕这些核心主题展开，帮助你全面了解如何将大模型高效、低成本地部署到实际业务场景中，并为如何搭建稳定的系统架构提供指导。下一节课将深入探讨如何为大模型的应用构建坚不可摧的合规壁垒与安全防线。

## 1. 业务需求分析
业务需求分析是成功部署大模型的第一步。不同的业务场景对模型的功能性需求和非功能性需求差异巨大。如果业务场景不清晰，可能会导致以下问题：
* 模型选择错误：选择了不适合特定任务的模型，导致性能不佳或资源浪费。
* 用户体验下降：无法满足用户对实时性、准确性等关键指标的需求。
* 成本失控：未根据业务需求优化模型规模和部署方案，导致高昂的计算和运维成本。

因此，明确业务场景后，你需要进一步围绕功能性需求和非功能性需求进行深入分析，并据此制定具体的部署方案。

### 1.1 模型的功能性需求
不同业务场景对模型的要求各不相同，以下是一些典型任务场景下的模型选择推荐：
* 自然语言处理：这是最常见的大模型应用场景之一，主要包括问答系统、文本生成、翻译、情感分析等任务。
    * 通用任务：如开放域问答、新闻摘要生成等，可以使用通用的大语言模型（如Qwen、GPT等）。
    * 特定领域任务：对于数学解题、法务咨询、医疗诊断等专业领域，建议选择经过领域微调的模型。例如：
        * 数学问题：可以选择专门针对数学任务优化的模型，如Qwen-Math。
        * 法务问题：可以选择面向法律领域训练的模型，如通义法睿。
        * 医疗诊断：需要高准确性和专业术语支持的模型，通常需要结合领域知识图谱或规则引擎。
* 视觉：包括图像分类、目标检测、图像生成等。这类任务通常需要专门的视觉模型（如通义万相、YOLO、Stable Diffusion等），而不是通用的语言模型。
* 语音：如语音助手、自动字幕生成、语音输入法、语音合成等。这类任务通常需要专门的语音处理模型（如Qwen-Audio、CosyVoice等）
* 多模态任务：结合文本、图像、视频、语音等多种模态，处理复杂任务。这里建议你使用专门设计的多模态模型（如Qwen-VL）能够显著提升效率和一致性，因为简单组合多个单模态模型（如语音识别 + 文本生成 + 图像理解）来完成多模态任务，会导致应用整体的延迟较高、一致性差、开发复杂度高等问题。

确定任务场景后，你可能还会发现有很多功能类似的大模型可供选择（如Qwen、DeepSeek、GPT等），推理的**准确度**可能是你选择模型的最主要原因，你需要**自己构建评测数据集**或**选择与业务场景匹配的公开数据集**进行评测（如MMLU评估语言理解、BBH测试复杂推理等）。

<a href="https://img.alicdn.com/imgextra/i2/O1CN01YFnJL820aE1wiLRgS_!!6000000006865-0-tps-2832-1118.jpg" target="_blank">
<img src="https://img.alicdn.com/imgextra/i2/O1CN01YFnJL820aE1wiLRgS_!!6000000006865-0-tps-2832-1118.jpg" width="900">
</a>

图片来源：[开源大模型评测排行](https://huggingface.co/spaces/open-llm-leaderboard/open_llm_leaderboard#/) （需要外网访问权限）

### 1.2 模型的非功能性需求

在正式地将模型应用于生产之前，除了选择更适合于业务场景的模型外（功能性需求），通常还需要关注**性能、成本、稳定**等方面（非功能性需求）。在本小节接下来的内容中，将分块展开讨论这些内容。

<br>



## 2. 性能优化

你的业务是否在意响应速度？例如，对话系统通常要求快速反馈（通常低于500ms），而离线批处理任务对延迟要求较低（最长可以是数小时甚至数天）。通常来说你需要明确**服务化级别目标 SLO（Service Level Object）**，这是云厂商提供LLM部署服务时给客户保证的一个性能指标，常见的指标有 **TTFT（Time to first token latency，首 token 延时）** 和 **TPOT（Time per output token，每 token 生成时间）**。不同的业务场景的可选性能评估数据集和 SLO 要求参考如下：

| 业务场景            | 常用性能评估数据集  | TTFT 要求  | TPOT 要求  |
|-------------------|------------------|------------|-----------|
| 对话、咨询、搜索类         | ShareGPT，MMLU | 高 | 中 |
| 代码补全、编程、网页设计    | HumanEval      | 高 | 高 |
| 阅读理解/总结/数据处理/信息提取 | LongBench | 低 | 中 |
| 通用大模型（DeepSeek R1，通义大模型等） | InfoVQA 等多模态评估数据集 | TTFT < 5sec（推荐小于该值） | TPOT < 200ms（推荐小于该值）|

当确定了系统所需要的 SLO 服务级别目标（包括TTFT要求和TPOT要求）后，就可以围绕这些目标对大模型应用进一步优化。

### 2.1 系统性能提升

下面列举了一些提升 LLM 系统性能的原则，这些原则是通用的，与业务无关，良好的应用这些原则可以帮助你从多个角度降低系统延迟，提升用户体验。

#### 2.1.1 更快地处理请求

通常影响模型推理速度的最主要因素是**模型的大小**，较小规模的模型可以更快地完成推理。

对于简单业务场景，你可以直接选择参数较小的模型（如Qwen2-7b），也可以通过模型压缩与量化的方式加速推理速度，常见的方法有：
* 模型剪枝：移除模型中冗余的权重或层，降低模型复杂度。
* 量化：使用 INT4、INT8 或 FP16 量化技术，减少模型推理所需的计算资源。
* 知识蒸馏：使用知识蒸馏技术训练小型模型，替代大型模型完成推理任务。

| 模型剪枝 | 量化 | 知识蒸馏 |
|--------|-----|---------|
| <img src="https://img.alicdn.com/imgextra/i2/O1CN01qWEBtx1gif0bta4YW_!!6000000004176-0-tps-1448-784.jpg" width="308"> | <img src="https://img.alicdn.com/imgextra/i1/O1CN01C4UG8o1Y18RZPZXFJ_!!6000000002998-2-tps-680-486.png" width="234"> | <img src="https://img.alicdn.com/imgextra/i2/O1CN018yxBtQ1FtCUwbMrY6_!!6000000000544-0-tps-1969-807.jpg" width="310"> <br> 图片来源：[Knowledge Distillation: A Survey](https://arxiv.org/pdf/2006.05525)|

在前面章节中，讨论过如何让较小的模型也能够高质量地完成推理，包括：
* 优化提示词：你可以通过提示词扩写来提供更详细的提示，也可以添加更多的样本来指引模型更好地推理。
* 微调模型：在特定领域/任务中，小模型微调后可能接近甚至超越大模型。

<br>

#### 2.1.2 减少大模型处理请求数和运算量

通过减少大模型处理的请求数量或运算量，可以降低硬件负载（如GPU）和并发压力，缩短排队和推理时间，从而降低系统整体延迟，提升性能。
* 上下文缓存：使用文本生成模型时，不同的推理请求之间可能会有重合的输入内容（如多轮对话、针对一本书的多次提问等），上下文缓存（Context Cache）技术可以将这些请求的公共前缀内容进行缓存，在推理时减少重复的运算量，提升响应速度。通义千问系列模型（qwen-max、 qwen-plus 、qwen-turbo）默认支持[上下文缓存（Context Cache）](https://help.aliyun.com/zh/model-studio/user-guide/context-cache)可以显著提升响应速度，并降低使用成本。
* 批处理：通过合并多个请求为一个批次（同时合并相似请求或去除重复请求），可以减少请求次数，降低多次请求间的往返延迟，提高硬件利用率。百炼上提供了[批量推理（Batch）](https://help.aliyun.com/zh/model-studio/user-guide/batch-inference)API，通过利用空闲时间资源完成**离线推理任务**，你可以通过这些接口执行批量推理任务。

<br>

#### 2.1.3 减少 Tokens 的输入和输出

在让大模型处理任务过程中，减少 token 的输入和输出可以帮助模型缩减推理时间，从而加快响应速度，这对于实时性要求较高的应用场景（如对话系统、客服机器人）尤为重要。
* 输入端优化：精简输入内容，去除输入中的冗余信息或无关内容，只保留关键信息。例如，在对话系统中，可以通过预处理提取用户的意图和核心问题，而不是将整个对话历史输入模型。还可以对长文档或复杂输入内容生成摘要，作为模型的输入。这可以通过小型摘要模型或规则方法实现。
* 输出端优化：相对于输入端优化，输出端的优化往往更为重要，因为生成 token 的过程几乎总是最耗时的。最简单的优化方法就是通过提示词（prompt）明确要求模型生成简洁的回答。例如，“请用一句话回答”或“请非常简短地解释”。同时在结构化输出的场景中，可以尽可能优化输出内容，如去除重复性描述、缩短函数名等。最后虽不常见，你还可以在通过 API 调用模型时，明确指定最大输出长度（max_tokens 参数），限制生成内容的规模。

<br>

#### 2.1.4 并行化处理

在大模型应用中，采用并行化处理可以有效提升计算效率、缩短推理或训练时间。通过将任务分解为多个子任务（如数据并行、模型并行或流水线并行），可以分别在不同的GPU或服务器上同时执行，从而减少整体耗时。

例如，数据并行通过对输入数据分片并分配到多个设备上处理，模型并行将模型的不同层或参数分布到不同设备，而流水线并行则将计算过程划分为阶段依次执行。这些方法能够显著降低单设备的计算压力，突破内存限制，并提高吞吐量，为大模型的高效部署和运行提供关键支持。

<br>

#### 2.1.5 不要默认依赖大模型

大语言模型（LLM）虽然功能强大且用途广泛，但并不意味着它适合处理所有任务。在某些情况下，默认使用 LLM 可能会导致不必要的延迟或复杂性，而更简单、经典的方法反而能够提供更好的性能和效率。以下是一些优化建议：
* 硬编码：减少对动态生成的依赖。如果输出是高度标准化或受限的，硬编码可能是更好的选择，而不是依赖 LLM 动态生成内容。例如：
    * 操作确认消息：像“您的请求已成功提交”或“操作失败，请重试”等标准响应可以直接硬编码，无需 LLM 生成。
    * 拒绝消息：像“输入无效，请检查格式”等常见错误场景，可以预先定义多种变体并随机选择，既高效又避免了重复感。
* 预先计算：提前生成和复用内容。当输入选项有限时，可以通过预先计算生成所有可能的响应，并根据用户输入快速匹配。这种方法不仅能减少延迟，还能避免重复显示相同的内容。
* 利用经典 UI 组件：提升用户体验。在某些场景下，传统的 UI 组件比 LLM 生成的文本更能有效地传达信息。例如：
    * 汇总指标：使用图表、进度条或表格来展示数据，而不是让 LLM 生成一段描述性文字。
    * 搜索结果：通过分页、筛选器和排序功能呈现结果，比生成一段冗长的自然语言描述更直观。
* 传统优化技术：结合经典算法提升效率。即使是在 LLM 应用中，经典的优化技术依然适用。例如：
    * 二分查找：在处理有序数据时，使用二分查找快速定位目标，而不是让 LLM 遍历整个数据集。
    - 哈希映射：通过哈希表快速检索预定义的响应或模板，减少计算复杂度。

### 2.2 用户感知优化

除了上面介绍的几种提升 LLM 系统性能的原则外，还可以通过以下方法进一步提升用户在使用过程中的满意度。

#### 2.2.1 流式输出

将生成的内容逐步返回给用户的技术，可以减少用户感知延迟、提高交互的流畅性，从而显著提升用户体验，尤其是在实时性要求较高的场景（如在线客服、语音助手）中。如果你的应用架构中使用了负载均衡，可能需要关闭缓存和数据压缩功能，避免流式输出失效。

<img src="https://img.alicdn.com/imgextra/i2/O1CN01xHhaED1FT5vHNVzEi_!!6000000000487-1-tps-954-720.gif" width="500">

<br>

#### 2.2.2 分块处理

将任务分解为多个小块，分别处理并逐步返回结果。比如在 RAG 系统中，分块处理可以应用于检索和生成两个阶段：
* 将检索任务分解为多个子任务，例如按主题或数据源分块检索。
* 将生成任务分解为多个段落或句子，分别生成并返回。

<br>

#### 2.2.3 展示任务进度

可以让用户了解当前系统的处理状态，减少因未知等待带来的焦虑感。在 RAG 系统中，可以通过进度条、加载动画或文字提示展示任务进度。例如：
* 在前端显示一个进度条，实时更新检索完成的比例。
* 显示生成进度（如“正在生成回答... 已完成 3/5 句话”）。

<br>

#### 2.2.4 完善错误处理机制

这是确保系统稳定性和用户体验的关键。明确的错误处理机制不仅能够捕获和处理各种异常情况，还能通过返回友好的错误信息和重试建议，提升用户对系统的信任感和满意度。
* 分类错误和友好提示：根据问题分类返回精简结果或默认响应，其中友好的错误信息提示是非常有必要的，
    * 清晰易懂：避免使用技术术语，确保普通用户也能理解。
    * 具体明确：说明错误原因，并提供解决方案。
    * 情感友好：语气温和，避免让用户感到挫败。
* 重试机制和降级：
    * 自动重试：对于临时性错误（如网络抖动、服务短暂不可用），可以自动重试。注意设置最大重试次数和重试间隔，设置过大的重试次数和过短的重试间隔可能会增加额外的资源消耗。
    * 错误降级：对于每种错误类型，设计相应的降级方案。如在返回错误信息的同时，提供重试按钮或操作指南。

<br>

#### 2.2.5 提供用户反馈入口与持续改进

* 反馈入口：在界面中提供便捷的反馈渠道，鼓励用户提出意见或报告问题。
* 持续优化：通过分析用户反馈及用户行为数据，持续发现问题并优化。

<br>

## 3. 成本优化

### 3.1 优化系统性能时节约成本

在 **2.1 系统性能提升** 中介绍的很多方法不仅能降低延迟，提升性能，还能帮助你有效节约成本，包括：
* **用小模型替换大模型** ：不仅推理更快，同时也更便宜。
* **上下文缓存** ：高频重复的查询，将结果缓存起来，避免每次调用 LLM 的开销。（如通义千问系列模型 cache_token 单价为 input_token 单价的40%）
* **批量推理** ：通过合并或去重请求，减少不必要的模型调用。一般来说批量推理任务对时效要求不高，可以通过离线推理方式充分利用空闲计算资源进一步降低成本。（如百炼批量推理的计费仅为实时推理的50%）
* **减少 token 数量** ：降低计算资源的需求，从而节省硬件成本和能源消耗。
* **不让大模型处理所有任务** ：通过硬编码、预先计算等方式替换大模型推理可以有效降低成本。

<br>

上面这些方法比较通用，在实际业务部署中，通常有两种方式：自建环境和云上部署。自建环境初期成本高，且服务器采购流程长、维护难度大。相比之下，云上部署更适合初创企业及对成本敏感的业务。云部署将基础设施维护等任务交由云厂商处理，并能根据业务需求快速调整资源，实现高效利用。

接下来的内容将帮助你了解如何在云上选型并选择合适的计费方式。

<br>

### 3.2 云上部署成本优化

由于大模型应用系统通常涉及大规模数据存储、高性能计算和复杂的模型推理，其运行成本可能较高。以下将详细探讨如何在云上基于大模型应用进行架构选型和成本优化。

#### 3.2.1 选择合适的GPU实例规格

在阿里云上部署大模型时，需要根据模型的性能要求和预算限制综合选择合适的 ECS 实例类型（如单机单卡、单机多卡或多机多卡）。这不仅需要考虑模型本身的显存占用，还需要综合评估模型运行时的其他资源需求（如 KV Cache 的设置情况）：
* **模型参数量** ：模型的参数量直接决定了其显存需求。在前面微调章节中介绍过，1.5B 参数的模型通常需要约 5.59GB 显存（FP32 精度），而 DeepSeek-R1（满血版 671B）的存储至少需要 $\frac {671×{10^9}} {2^{30}}≈ 625GB$ 显存（FP8精度）。
* **KV Cache 占用** ：对于生成式任务（如文本生成或对话系统），KV Cache 是存储注意力机制中键值对缓存的空间。KV Cache 的大小与输入序列长度和批次大小成正比。例如，处理长上下文（如 2048 tokens）时，KV Cache 可能会占用大量显存。
* **精度设置** ：不同的计算精度（如 FP32、FP16）会影响显存需求。使用量化技术（如 INT8 或 INT4）可以显著减少显存占用。

##### **如何根据模型选择合适的GPU实例？**

以 DeepSeek-R1（671B）为例，模型本身预估占用显存 625GB（FP8 精度），KV Cache 使用 MLA（Multi-head Latent Attention） 优化，假设所有层都使用 MLA ，那么单卡下每个 token 预计占用的显存为 70KB，一个 64K token 长度请求在 8卡环境下 KV Cache 预估占用显存 35G，总共占用 660 GB，刚好可以选择 ecs.ebmgn8v.48xlarge 实例规格（实例显存 8*96 GB）。

##### **用户并发如何计算？**

还是以上面的例子来预估，可以处理的并发请求数和 Token 长度满受显存大小限制如下（不考虑内存对齐、ECC、框架运行时的显存占用）：

$\frac{req * l * (70 * 1024) }{2^{30}} + \frac{625}{8} < 96GB$

$req$表示并发请求数， $l$表示 token 长度。

假设 $l=64K$，可推算出 $req≈4$ 可见由于显存的局促导致同时服务客户的能力非常有限，如果需要承载更高并发可以适当限制单次请求的token长度或选择更高规格的GPU实例。

<br>

#### 3.2.2 选择合适的计费方式

确定GPU实例规格后，你要根据业务场景需求，选择合适的计费方式：

|  | [预付费（包年包月）](https://help.aliyun.com/zh/ecs/subscription) | [按量付费](https://help.aliyun.com/zh/ecs/pay-as-you-go-1) | [抢占式实例（spot）](https://help.aliyun.com/zh/ecs/preemptible-instance) |
|----|----|----|----|
| **说明** | 直接预支一笔费用，提前预留资源并享受更大的价格优惠。 | 按需开通和释放资源，无需提前支付费用，灵活快速。<br>短期运行成本较低，中长期使用资源可结合[节省计划](https://help.aliyun.com/zh/ecs/savings-plans)降低资源使用成本。 | 先使用后付费，市场价格根据供需关系实时变化，相对于按量付费最高能节约90%的成本。 |
| **推荐场景** | 适用于**相对稳定**的业务场景部署（如数月或数年）| 适用于**非固定时长**的业务场景部署 | 适用于非固定时长且**追求极致的成本控制**的业务场景部署，如非关键业务即使偶尔服务中断也不会造成重大损失。通常需结合重试机制或其他容错手段来应对短暂的服务不可用。 <br> `基于抢占式实例的在线推理服务部署方案：`[`PAI-EAS Spot最佳实践`](https://help.aliyun.com/zh/pai/use-cases/pai-eas-spot-best-practices) |

同时还建议你使用[预算管理](https://billing-cost.console.aliyun.com/expense-manage/expense-budget/list)设置预算告警，并定期通过[成本分析](https://billing-cost.console.aliyun.com/expense-manage/expense-analyze)识别高成本模块（如GPU/ECS实例）。测试不同配置的性价比，高负载时优先保障核心功能（如检索），关闭非必要服务（如复杂生成任务）。

<br>

## 4. 稳定性

前面内容中我们对于如何评估和优化模型性能，以及通过云服务进一步降低模型的运行成本有了比较深度的探讨，但是在将你的大模型应用投入生产时，必须保证系统能够提供稳定可靠的服务，这就像一家24小时营业的便利店，无论何时去都能正常购物。如果模型服务频繁“关门”或“反应迟钝”，用户会失去信任，企业也可能面临损失，通过以下方法可以有效保障你的模型应用的线上稳定性：

### 4.1 降低用户请求的资源消耗

在性能和成本优化中讲到的一些方法同样有助于提升系统稳定性，比如通过模型**小型化处理、异步批处理、缓存高频结果**等方式可以有效降低用户请求消耗的资源，从而使得相同配置的资源可以承担更多用户的请求，间接提升高并发场景下的稳定性。

<br>

### 4.2 自动化扩缩容

通用云应用的高可用架构同样适用于 LLM 系统，你可以根据用户请求数量自动增加或减少计算资源，在保证服务稳定的前提下避免资源浪费：
* **水平伸缩计算资源** ：[弹性伸缩（ESS）](https://help.aliyun.com/zh/auto-scaling/use-cases/deploy-a-highly-available-computing-cluster-using-a-balanced-distribution-strategy)动态调整ECS/GPU实例数量，或使用[函数计算（FC）](https://www.aliyun.com/product/fc)按需分配资源。
* **分散流量压力** ：通过[负载均衡（SLB）](https://www.aliyun.com/product/slb)提升高并发场景下的处理能力。可参考：[通过ALB将用户请求分配到多台ECS服务器](https://help.aliyun.com/zh/slb/application-load-balancer/getting-started/use-an-alb-instance-to-provide-ipv4-services)、[ALB添加函数计算FC作为后端服务](https://help.aliyun.com/zh/slb/application-load-balancer/use-cases/specify-a-function-from-function-compute-as-a-backend-server-of-alb)

<br>

### 4.3 评测基线管理

评测基线就像一把“标尺”，用来衡量模型的好坏，避免盲目优化后降低系统稳定性，同时也为后续的容灾降级（如切换至基线模型）提供可靠的后备方案。以下是建立评测基线的关键方法：
1. **建立基线模型**
    * **从简单开始** ：用基础算法（如决策树）或预设规则（如关键词匹配）作为初始模型，快速验证可行性。如：客服系统先用“关键词匹配”判断用户问题，准确率70%作为基线，后续优化需超过它。
    * **历史版本参考** ：将旧模型的性能设为最低门槛，新模型必须更好才能上线。
2. **定期测试与对比**
    * **时间维度** ：定期（如每周）用最新数据测试新旧模型，防止性能下降。如：电商推荐系统若点击率低于基线，说明模型需调整。
    * **场景维度** ：针对不同情况（如促销高峰期、不同地区）设置多组基线，全面评估稳定性。
3. **动态调整基线**
    * **应对数据变化** ：当业务或用户行为变化时，重新训练基线。如：金融风控政策调整后，需更新欺诈检测基线。
    * **适配业务需求** ：如更关注速度而非精度，改用轻量级模型（如蒸馏模型）作为新基线。
4. **融入自动化流程**
    * **自动拦截不合格模型** ：在部署流程中加入基线测试，未达标版本直接阻断。
    * **小范围试用新模型** ：先让5%用户试用新模型，效果达标再全面替换，即灰度发布。

<br>

### 4.4 模型实时监控与预警

* **关键指标看板** ：监控模型准确率、响应速度（如“每秒处理多少请求”）、错误率等，像“汽车仪表盘”一样实时显示健康状态。
* **数据漂移检测** ：对比当前输入数据与训练数据的分布差异。例如，若用户突然大量提问“如何退货”，系统会提示“数据分布异常，可能需重新训练模型”。
* **自动告警与日志追踪** ：设置阈值（如响应时间超过2秒将自动报警），并记录每条请求的输入、输出和上下文，方便快速定位问题。

<br>

### 4.5 容灾性设计

大模型应用的容灾设计可以想象成给系统准备“备用工具”和“应急计划”。良好的容灾设计可以应对突发故障，如模型响应异常、服务器宕机、网络中断或自然灾害（如地震）时，确保服务快速恢复：
* **降级与熔断机制**：当检测到模型响应异常（如准确率骤降或延迟超限），自动触发降级策略。如切换至备份模型（如上一稳定版本）或启用规则引擎兜底（如预设固定回复模板）。
* **通用应用容灾方案**同样适用于 LLM 系统：你可以跨地域跨可用区部署大模型应用，提升系统的可靠性。同时如果成本允许情况下，还可以创建备用环境，实现故障后的快速切换。如果对成本相对敏感，你的业务对系统恢复时间（如RTO和RPO）较为宽容，还可以通过快速创建环境的方式恢复业务。
* **定期演练预测试**：还建议你定期模拟故障场景（如模型异常、断网、服务器宕机等），验证容灾方案是否有效。

<br>


## ✅ 本节小结

在本节课程中，我们学习了以下内容：
- **大模型应用发布至生产环境的关键要素** ，包括业务需求分析、模型的功能性需求（如自然语言处理、视觉、语音等场景的模型选择）与非功能性需求（性能、成本、稳定等）
- **性能与成本优化策略** ，如模型压缩（剪枝、量化、知识蒸馏）、请求批处理、缓存机制、输入输出Token优化及并行化处理
- **用户体验提升方法** ，包括流式输出、分块处理、任务进度展示、错误处理机制及用户反馈闭环设计
- **稳定性提升方法** ，包括自动化扩缩容、模型实时监控与预警以及通过容灾设计保障业务稳定运行
- **云上架构选型** ，涵盖GPU实例规格选择、多卡推理优化、数据传输加密

除了本节课程中的示例展示的内容之外，你还可以尝试：
- **多卡PD分离提升模型推理效率** ：在大模型（如语言模型）的推理过程中，Prefill（预填充阶段） 和 Decode（解码阶段） 是两个关键阶段，分别对应输入处理和输出生成的过程。在多卡环境下，通过分解 Prefill 和 Decode 过程可以优化模型的推理速度，从而满足 TTFT（首 Token 延迟） 和 TPOT（每 Token 生成时间） 的 SLO 需求。[DistServe](https://github.com/LLMServe/DistServe) 是分离 Prefill/Decode 的典型系统实现，结合分块预填充和异步流水线，显著提升吞吐并降低延迟。
- **引入 MLOps 管理 AI 系统** ：MLOps（Machine Learning Operations）是机器学习与 DevOps 的结合，旨在通过自动化、标准化和协作的方式管理机器学习模型的全生命周期。它借鉴了软件工程中的 DevOps 理念，将开发（Development）和运维（Operations）紧密结合，确保机器学习模型能够高效地从开发阶段过渡到生产环境，并在生产环境中持续优化和迭代。

在学习本课程的时候，如果你想了解更多相关概念和原理，可以尝试进一步学习如下内容或通过大模型给出学习建议，如：
- **KV Cache优化实践** ：深入理解KV Cache的作用及优化方法。

## ✉️ 评价反馈
感谢你学习阿里云大模型ACP认证课程，如果你觉得课程有哪里写得好、哪里写得不好，期待你[通过问卷提交评价和反馈](https://survey.aliyun.com/apps/zhiliao/Mo5O9vuie)。

你的批评和鼓励都是我们前进的动力。